In [22]:
# If you want to save a subset of the data, define these manually here. All three variables have to be defined. Else, leave this blank

import os 
# session_list = np.array([['LPE10885', '2023_10_20']])
# session_list = np.array(session_list)
# folders = [os.path.join(INPUT_FOLDER, 'LPE10885')]
# files = [[folder, os.path.join(folder, '2023_10_20')]
#          for folder in folders]

# Imports
# -*- coding: utf-8 -*-
"""
This script analyzes neural and behavioral data in a multi-area calcium imaging
dataset with labeled projection neurons. The visual stimuli are natural images.
Matthijs Oude Lohuis, 2023, Champalimaud Center
Anastasia Simonoff, 2024, Bernstein Center for Computational Neuroscience Berlin
"""

# Import general libs
import logging
from sklearn.preprocessing import normalize
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from tqdm.auto import tqdm
import sys

# Set working directory to root of repo
current_path = os.getcwd()
# Identify if path has 'molanalysis' as a folder in it
if 'molanalysis' in current_path:
    # If so, set the path to the root of the repo
    current_path = current_path.split('molanalysis')[0] + 'molanalysis'
else:
    raise FileNotFoundError(
        f'This needs to be run somewhere from within the molanalysis folder, not {current_path}')
os.chdir(current_path)
sys.path.append(current_path)

from sensorium_utility_training_read_config import read_config

run_config = read_config('../Petreanu_MEI_generation/run_config.yaml') # Must be set

RUN_NAME = run_config['RUN_NAME'] # MUST be set. Creates a subfolder in the runs folder with this name, containing data, saved models, etc. IMPORTANT: all values in this folder WILL be deleted.

keep_behavioral_info = run_config['data']['keep_behavioral_info']
area_of_interest = run_config['data']['area_of_interest']
sessions_to_keep = run_config['data']['sessions_to_keep']
OUTPUT_NAME = run_config['data']['OUTPUT_NAME']
INPUT_FOLDER = run_config['data']['INPUT_FOLDER']
OUTPUT_FOLDER = f'../molanalysis/MEI_generation/data/{OUTPUT_NAME}' # relative to molanalysis root folder

# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(pathname)s - %(message)s', handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
# Create a StreamHandler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Set the logging level for the handler
# Create a Formatter and attach it to the handler
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(pathname)s - %(message)s')
console_handler.setFormatter(formatter)
# Add the handler to the logger
logger.addHandler(console_handler)

logger.info('Current working directory: %s', os.getcwd())

# TODO: Fix this so it outputs correctly during figure generation
rmap_logger = logging.getLogger('rastermap')
rmap_logger.setLevel(logging.WARNING)
rmap_logger.addHandler(console_handler)
rmap_logger.propagate = False

# Import personal lib funcs
from loaddata.session_info import load_sessions
from utils.imagelib import load_natural_images
from utils.explorefigs import *
from loaddata.get_data_folder import get_local_drive


# Updated by Anastasia Simonoff for her local computer, etc. This should be updated for your local computer, too.

savedir = os.path.join(get_local_drive(
), 'Users\\asimo\\Documents\\BCCN\\Lab Rotations\\Petreanu Lab\\Figures\\Images' if os.environ['USERDOMAIN'] == 'ULTINTELLIGENCE' else 'OneDrive\\PostDoc\\Figures\\Images\\')
logger.info(f'Saving figures to {savedir}')

# INPUT_FOLDER = "../sensorium/notebooks/data/IM_prezipped"
# Add Add folders two levels deep from INPUT_FOLDER into a list

# Delete anything in OUTPUT_FOLDER
# import shutil
# try:
#     shutil.rmtree(OUTPUT_FOLDER)
# except FileNotFoundError:
#     os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# test if folders already defined 
try: 
    folders
except NameError:
    # First level
    folders = [os.path.join(INPUT_FOLDER, name) for name in os.listdir(
        INPUT_FOLDER) if os.path.isdir(os.path.join(INPUT_FOLDER, name)) and not "merged_data" in name]
    folders = [x.replace("\\", "/") for x in folders]
    # Second level
    files = [[folder, os.path.join(folder, name).replace('\\', '/')] for folder in folders for name in os.listdir(
        folder) if os.path.isdir(os.path.join(folder, name)) and not "merged_data" in name]
    # only get last value after /
    session_list = [[folder.split("/")[-1], name.split("/")[-1]]
                    for folder, name in files]

    # drop ['LPE10919', '2023_11_08'] because the data is not converted yet
    session_list = [x for x in session_list if x != ['LPE10919', '2023_11_08']]
    print(session_list)

if sessions_to_keep != 'all':
    session_list = [x for x in session_list if x in np.array(sessions_to_keep)]

# Load one session including raw data: ################################################
# example session with good responses

# Load sessions lazy: (no calciumdata, behaviordata etc.,)
sessions, nSessions = load_sessions(protocol='IM', session_list=np.array(session_list), data_folder=INPUT_FOLDER)

# Load proper data and compute average trial responses:
for ises in tqdm(range(nSessions)):    # iterate over sessions

    # os.makedirs(os.path.join(OUTPUT_FOLDER, session_list[ises][0], session_list[ises][1], 'data'), exist_ok=True)

    sessions[ises].load_respmat(calciumversion='deconv', keepraw=True)

    # Save respmat
    # # np.save(os.path.join(files[ises][1], 'respmat.npy'), sessions[ises].respmat)
    # np.save(os.path.join(OUTPUT_FOLDER, session_list[ises][0], session_list[ises][1], 'data', 'respmat.npy'), sessions[ises].respmat)

# Load all IM sessions including raw data: ################################################
# sessions,nSessions   = filter_sessions(protocols = ['IM'])
# for ises in range(nSessions):    # iterate over sessions
#     sessions[ises].load_respmat(calciumversion='deconv',keepraw=False)

def replace_nan_with_avg(arr):
    arr = arr.copy()  # Copy the array to avoid modifying the original
    nan_indices = np.where(np.isnan(arr))[0]  # Get indices of NaN values

    for i in nan_indices:
        # Handle cases where NaN is at the start or end of the array
        if i == 0:
            arr[i] = arr[i + 1]
        elif i == len(arr) - 1:
            arr[i] = arr[i - 1]
        else:
            # Replace NaN with the average of adjacent values
            arr[i] = np.nanmean([arr[i - 1], arr[i + 1]])

    return arr

import shutil 
# Save behavior data in sensorium format

idx_to_delete = []

for i, (sess, sess_obj) in enumerate(zip(session_list, sessions)):
    folder_base = f'{OUTPUT_FOLDER}/{sess[0]}/{sess[1]}'
    pupil_size = sess_obj.respmat_pupilarea.reshape(-1, 1)
    change_of_pupil_size = sess_obj.respmat_pupilareaderiv.reshape(-1, 1)
    locotomion_speed = sess_obj.respmat_runspeed.reshape(-1, 1)

    # Data
    folder = f'{folder_base}/data/behavior'
    # os.makedirs(folder, exist_ok=True)

    # if np.isnan(pupil_size).all() or np.isnan(change_of_pupil_size).all() or np.isnan(locotomion_speed).all():
    #     logger.warning(
    #         f'All values in behavior data for session {sess[0]}/{sess[1]} are NaN. Session will be removed.')
    #     # Drop session from list
    #     # session_list = np.delete(session_list, i, axis=0)
    #     # sessions = np.delete(sessions, i, axis=0)
    #     idx_to_delete.append(i)
    #     # Remove folder
    #     shutil.rmtree(folder_base)
    #     try:
    #         os.rmdir(folder_base)
    #     except FileNotFoundError:
    #         pass
    #     try:
    #         shutil.rmtree(f'../sensorium/notebooks/data/IM_prezipped/{sess[0]}/{sess[1]}/')
    #         if len(os.listdir(f'../sensorium/notebooks/data/IM_prezipped/{sess[0]}')) == 0:
    #             os.rmdir(f'../sensorium/notebooks/data/IM_prezipped/{sess[0]}/')
    #     except FileNotFoundError:
    #         pass
    #     continue

    behavior = np.hstack((pupil_size, change_of_pupil_size, locotomion_speed))
    if keep_behavioral_info:
        behavior = replace_nan_with_avg(behavior)
        assert not np.isnan(behavior).any(), 'There are still NaN values in the behavior data'
    else:
        behavior = np.random.default_rng().normal(size=behavior.shape)

    # for i in tqdm(range(behavior.shape[0])):
        # np.save(f'{folder}/{i}.npy', behavior[i, :])

    # Meta
    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/behavior/all'
    # os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(behavior, axis=0)
    # np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(behavior, axis=0)
    # np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(behavior, axis=0)
    # np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(behavior, axis=0)
    # np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(behavior, axis=0)
    # np.save(f'{folder}/median.npy', median)

    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/behavior/stimulus_Frame'
    # os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(behavior, axis=0)
    # np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(behavior, axis=0)
    # np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(behavior, axis=0)
    # np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(behavior, axis=0)
    # np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(behavior, axis=0)
    # np.save(f'{folder}/median.npy', median)

logger.warning(f'Removing sessions idx = {idx_to_delete} from sessions lists.')
session_list = np.delete(session_list, idx_to_delete, axis=0)
sessions = np.delete(sessions, idx_to_delete, axis=0)

### Load the natural images:
# natimgdata = load_natural_images(onlyright=True)
natimgdata = load_natural_images()
natimgdata = natimgdata[:, natimgdata.shape[1]//2:, :]  # Only take the right half

# Save natimgdata in sensorium format

for sess, sess_obj in zip(session_list, sessions):
    folder_base = f'{OUTPUT_FOLDER}/{sess[0]}/{sess[1]}'

    folder = f'{folder_base}/data/images'
    # os.makedirs(folder, exist_ok=True)

    image_idxs = sess_obj.trialdata['ImageNumber'].values

    for i, imgidx in tqdm(enumerate(image_idxs), total=len(image_idxs)):
        file_name = f'{folder}/{i}.npy'
        img = natimgdata[:, :, imgidx]
        img = np.reshape(img, (-1, img.shape[0], img.shape[1]))
        # np.save(file_name, img)  # hacky but works

    print(f'num_trials for {sess[0]}/{sess[1]}: {len(image_idxs)}')

    # Meta
    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/images/all'
    # os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(natimgdata)
    # np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(natimgdata)
    # np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(natimgdata)
    # np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(natimgdata)
    # np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(natimgdata)
    # np.save(f'{folder}/median.npy', median)

    # Meta
    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/images/stimulus_Frame'
    # os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(natimgdata)
    # np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(natimgdata)
    # np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(natimgdata)
    # np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(natimgdata)
    # np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(natimgdata)
    # np.save(f'{folder}/median.npy', median)

# Save pupil center data in sensorium format

for sess, sess_obj in zip(session_list, sessions):
    folder_base = f'{OUTPUT_FOLDER}/{sess[0]}/{sess[1]}'
    folder = f'{folder_base}/data/pupil_center'
    # os.makedirs(folder, exist_ok=True)
    pupil_x = sess_obj.respmat_pupilx.reshape(-1, 1)
    pupil_y = sess_obj.respmat_pupily.reshape(-1, 1)

    pupil_center = np.hstack((pupil_x, pupil_y))

    if keep_behavioral_info:
        pupil_center = replace_nan_with_avg(pupil_center)

        while np.isnan(pupil_center).any():
            pupil_center = replace_nan_with_avg(pupil_center)
    else:
        pupil_center = np.random.default_rng().normal(size=pupil_center.shape)

    print(f'num_trials for {sess[0]}/{sess[1]}: {len(pupil_center)}')

    # for i in tqdm(range(pupil_center.shape[0])):
        # np.save(f'{folder}/{i}.npy', pupil_center[i, :])

    # Meta
    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/pupil_center/all'
    # os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(pupil_center, axis=0)
    # np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(pupil_center, axis=0)
    # np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(pupil_center, axis=0)
    # np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(pupil_center, axis=0)
    # np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(pupil_center, axis=0)
    # np.save(f'{folder}/median.npy', median)

    # Meta
    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/pupil_center/stimulus_Frame'
    # os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(pupil_center, axis=0)
    # np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(pupil_center, axis=0)
    # np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(pupil_center, axis=0)
    # np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(pupil_center, axis=0)
    # np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(pupil_center, axis=0)
    # np.save(f'{folder}/median.npy', median)

# Add neuron data

for sess, sess_obj in zip(session_list, sessions):
    folder_base = f'{OUTPUT_FOLDER}/{sess[0]}/{sess[1]}'
    folder = f'{folder_base}/meta/neurons'
    # os.makedirs(folder, exist_ok=True)

    celldata = sess_obj.celldata.copy()

    # layer
    # V1
    celldata.loc[(celldata['roi_name'] == 'V1') & (
        celldata['depth'] < 250), 'layer'] = 'L2/3'
    celldata.loc[(celldata['roi_name'] == 'V1') & (
        celldata['depth'] >= 250) & (celldata['depth'] < 350), 'layer'] = 'L4'
    celldata.loc[(celldata['roi_name'] == 'V1') & (
        celldata['depth'] >= 350), 'layer'] = 'L5/6'

    # PM
    celldata.loc[(celldata['roi_name'] == 'PM') & (
        celldata['depth'] < 250), 'layer'] = 'L2/3'
    celldata.loc[(celldata['roi_name'] == 'PM') & (
        celldata['depth'] >= 250) & (celldata['depth'] < 325), 'layer'] = 'L4'
    celldata.loc[(celldata['roi_name'] == 'PM') & (
        celldata['depth'] >= 325), 'layer'] = 'L5'
    
    celldata = celldata.loc[celldata['roi_name'] == area_of_interest] if area_of_interest is not None else celldata
    
    # Save celldata to obj
    sess_obj.celldata = celldata.copy()

    num_neurons = len(celldata)

    print(f'num_neurons for {sess[0]}/{sess[1]}: {num_neurons}')
    
    # layer
    # np.save(f'{folder}/layer.npy',
    #         celldata['layer'].to_numpy(dtype='<U32'))
    
    # animal ids
    # np.save(f'{folder}/animal_ids.npy',
    #         np.full((num_neurons, ), sess_obj.animal_id, dtype='<U32'))

    # area
    # np.save(f'{folder}/area.npy',
    #         celldata['roi_name'].to_numpy(dtype='<U32'))

    # cell motor coordinates
    # np.save(f'{folder}/cell_motor_coordinates.npy',
    #         celldata[['xloc', 'yloc', 'depth']].to_numpy(dtype=int))

    # scan idx
    # np.save(f'{folder}/scan_idx.npy',
    #         np.full((num_neurons, ), 0))

    # sessions
    # np.save(f'{folder}/sessions.npy',
    #         celldata['session_id'].to_numpy(dtype='<U32'))

    # unit ids
    # np.save(f'{folder}/unit_ids.npy',
    #         celldata['cell_id'].to_numpy(dtype='<U32'))

# Save responses in sensorium format

for sess, sess_obj in zip(session_list, sessions):
    folder_base = f'{OUTPUT_FOLDER}/{sess[0]}/{sess[1]}'
    folder = f'{folder_base}/data/responses'
    # os.makedirs(folder, exist_ok=True)

    responses = sess_obj.respmat
    responses = replace_nan_with_avg(responses)

    celldata = sess_obj.celldata.copy()

    responses = responses[celldata.index.values]

    sess_obj.respmat = responses

    print(f'num_neurons for {sess[0]}/{sess[1]}: {responses.shape[0]}')

    # for i in tqdm(range(responses.shape[1])):
        # np.save(f'{folder}/{i}.npy', responses[:, i])

    # Meta
    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/responses/all'
    # os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(responses, axis=1)
    # np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(responses, axis=1)
    # np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(responses, axis=1)
    # np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(responses, axis=1)
    # np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(responses, axis=1)
    # np.save(f'{folder}/median.npy', median)

    # Meta
    # There is also a /stimulus_Frame, but I'm not sure what the difference is
    folder = f'{folder_base}/meta/statistics/responses/stimulus_Frame'
    # os.makedirs(folder, exist_ok=True)

    # Mean
    mean = np.mean(responses, axis=1)
    # np.save(f'{folder}/mean.npy', mean)

    # Std
    std = np.std(responses, axis=1)
    # np.save(f'{folder}/std.npy', std)

    # Min
    min = np.min(responses, axis=1)
    # np.save(f'{folder}/min.npy', min)

    # Max
    max = np.max(responses, axis=1)
    # np.save(f'{folder}/max.npy', max)

    # Median
    median = np.median(responses, axis=1)
    # np.save(f'{folder}/median.npy', median)

def calculate_tiers(num_images):
    # Split into train/test/validate
    idxs = np.arange(num_images)
    np.random.shuffle(idxs)
    train_idxs = idxs[:int(num_images * 0.75)]
    test_idxs = idxs[int(num_images * 0.75):int(num_images * 0.916)]
    validate_idxs = idxs[int(num_images * 0.916):]
    return train_idxs, test_idxs, validate_idxs

# Add trial data


for sess, sess_obj in zip(session_list, sessions):
    folder_base = f'{OUTPUT_FOLDER}/{sess[0]}/{sess[1]}'
    folder = f'{folder_base}/meta/trials'
    # os.makedirs(folder, exist_ok=True)

    trial_data = sess_obj.trialdata.copy()
    num_trials = trial_data.shape[0]

    print(f'num_trials for {sess[0]}/{sess[1]}: {num_trials}')

    # album
    # ???
    # np.save(f'{folder}/album.npy',
    #         np.full((num_trials,), 'UNK', dtype='<U32'))

    # animal id
    # np.save(f'{folder}/animal_id.npy',
    #         np.full((num_trials,), sess_obj.animal_id, dtype='<U32'))

    # condition hash
    # ???
    # np.save(f'{folder}/condition_hash.npy',
    #         np.full((num_trials,), 'UNK', dtype='<U32'))

    # frame image class
    # ???
    # np.save(f'{folder}/frame_image_class.npy',
    #         np.full((num_trials,), 'UNK', dtype='<U32'))

    # frame image id
    # np.save(f'{folder}/frame_image_id.npy', trial_data['ImageNumber'].values)

    # frame last flip
    # ???
    # np.save(f'{folder}/frame_last_flip.npy',
    #         np.full((num_trials,), 0))

    # frame pre blank period
    trial_data['tOnset'] = pd.to_datetime(trial_data['tOnset'], unit='s')
    trial_data['tOffset'] = pd.to_datetime(trial_data['tOffset'], unit='s')
    trial_data['presentationTime'] = trial_data['tOffset'] - \
        trial_data['tOnset']

    # calculate inter image interval
    trial_data['tOffset_prev'] = trial_data['tOffset'].shift(1)
    trial_data['tIntertrial'] = trial_data['tOnset'] - \
        trial_data['tOffset_prev']
    trial_data['tIntertrial'].fillna(pd.Timedelta(seconds=0.5), inplace=True)
    trial_data['tIntertrial'] = trial_data['tIntertrial'].dt.total_seconds()
    # np.save(f'{folder}/frame_pre_blank_period.npy',
    #         trial_data['tIntertrial'].values)

    # frame presentation time
    # np.save(f'{folder}/frame_presentation_time.npy',
    #         trial_data['presentationTime'].dt.total_seconds())

    # frame_trial_ts
    # np.save(f'{folder}/frame_trial_ts.npy', trial_data['tOnset'].apply(
    #     lambda x: f"Timestamp('{x}')").to_numpy(dtype='<U32'))

    # scan_idx
    # ???
    # np.save(f'{folder}/scan_idx.npy',
    #         np.full((num_trials,), 0))

    # session
    # np.save(f'{folder}/session.npy',
    #         np.full((num_trials,), sess_obj.session_id))

    # tiers
    trial_data['ImageCount'] = trial_data['ImageNumber'].map(
        trial_data['ImageNumber'].value_counts())
    trial_data['ImagePresentation'] = trial_data.groupby(
        'ImageNumber').cumcount() + 1

    # Assign tiers to images
    for i, group in trial_data.groupby('ImageCount'):
        for j, group2 in group.groupby('ImagePresentation'):
            train_idxs, test_idxs, validate_idxs = calculate_tiers(
                group2.shape[0])

            # Update indices
            train_idxs = group2.index[train_idxs]
            test_idxs = group2.index[test_idxs]
            validate_idxs = group2.index[validate_idxs]

            # assign to tiers
            trial_data.loc[train_idxs, 'tiers'] = 'train'
            trial_data.loc[test_idxs, 'tiers'] = 'test'
            trial_data.loc[validate_idxs, 'tiers'] = 'validation'

    # np.save(f'{folder}/tiers.npy',
    #         trial_data['tiers'].to_numpy(dtype='<U32'))

    # trial_idx
    # np.save(f'{folder}/trial_idx.npy', trial_data['TrialNumber'].values)

2024-12-15 22:32:53,776 - __main__ - INFO - C:\Users\asimo\AppData\Local\Temp\ipykernel_3536\3599183259.py - Current working directory: c:\Users\asimo\Documents\BCCN\Lab Rotations\Petreanu Lab\molanalysis
2024-12-15 22:32:53,776 - __main__ - INFO - C:\Users\asimo\AppData\Local\Temp\ipykernel_3536\3599183259.py - Current working directory: c:\Users\asimo\Documents\BCCN\Lab Rotations\Petreanu Lab\molanalysis
2024-12-15 22:32:53,776 - __main__ - INFO - C:\Users\asimo\AppData\Local\Temp\ipykernel_3536\3599183259.py - Current working directory: c:\Users\asimo\Documents\BCCN\Lab Rotations\Petreanu Lab\molanalysis
2024-12-15 22:32:53,776 - __main__ - INFO - C:\Users\asimo\AppData\Local\Temp\ipykernel_3536\3599183259.py - Current working directory: c:\Users\asimo\Documents\BCCN\Lab Rotations\Petreanu Lab\molanalysis
2024-12-15 22:32:53,776 - __main__ - INFO - C:\Users\asimo\AppData\Local\Temp\ipykernel_3536\3599183259.py - Current working directory: c:\Users\asimo\Documents\BCCN\Lab Rotations\

  0%|          | 0/1 [00:00<?, ?it/s]

Computing average response for response matrix:   0%|          | 0/5600 [00:00<?, ?it/s]

Computing average response for runspeed:   0%|          | 0/5600 [00:00<?, ?it/s]

Computing average response for motion energy:   0%|          | 0/5600 [00:00<?, ?it/s]

...

Computing average response for pupil y position:   0%|          | 0/5600 [00:00<?, ?it/s]

Computing average response for pupil area:   0%|          | 0/5600 [00:00<?, ?it/s]

2024-12-15 22:33:38,505 - utils.explorefigs - WARNING - C:\Users\asimo\AppData\Local\Temp\ipykernel_3536\3599183259.py - Removing sessions idx = [] from sessions lists.


  0%|          | 0/5600 [00:00<?, ?it/s]

num_trials for LPE10885/2023_10_20: 5600
num_trials for LPE10885/2023_10_20: 5600
num_neurons for LPE10885/2023_10_20: 1173
num_neurons for LPE10885/2023_10_20: 1173
num_trials for LPE10885/2023_10_20: 5600


In [23]:
celldata

,iscell,iscell_prob,skew,radius,npix_soma,npix,xloc,...,layer,recombinase,session_id,rf_az_F,rf_el_F,rf_sz_F,rf_p_F
0,1.0,0.967153,5.268604,6.838876,167.0,383.0,254.0,...,L2/3,non,LPE10885_2023_10_20,119.944238,34.223881,28.0,0.001149
1,1.0,0.996898,7.369462,7.338488,141.0,190.0,406.0,...,L2/3,non,LPE10885_2023_10_20,-27.602230,12.256716,17.0,0.153754
2,1.0,0.989760,3.229899,5.819851,125.0,150.0,490.0,...,L2/3,non,LPE10885_2023_10_20,43.661710,14.253731,33.0,0.088756
3,1.0,0.991979,7.171705,5.773342,117.0,145.0,210.0,...,L2/3,non,LPE10885_2023_10_20,108.903346,0.274627,6.0,0.017176
4,1.0,0.980091,7.256192,6.157345,133.0,176.0,262.0,...,L2/3,non,LPE10885_2023_10_20,123.959108,19.246269,169.0,0.000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,1.0,0.920875,0.889110,5.487384,79.0,93.0,465.0,...,L4,non,LPE10885_2023_10_20,71.765799,15.252239,2.0,0.015785
1169,1.0,0.622431,1.244387,4.368339,64.0,79.0,139.0,...,L4,non,LPE10885_2023_10_20,80.799257,20.244776,8.0,0.024745
1170,1.0,0.684631,1.711489,6.102885,70.0,77.0,201.0,...,L4,non,LPE10885_2023_10_20,83.810409,12.256716,3.0,0.083206
1171,1.0,0.976950,2.105739,6.052476,79.0,93.0,493.0,...,L4,non,LPE10885_2023_10_20,45.669145,18.247761,16.0,0.231020


```
num_neurons for LPE09665/2023_03_20: 563
num_neurons for LPE10883/2023_10_23: 854
num_neurons for LPE10883/2023_10_31: 1157
num_neurons for LPE10884/2023_10_12: 1071
num_neurons for LPE10885/2023_10_20: 1173
num_neurons for LPE10919/2023_11_09: 1735
num_neurons for LPE11086/2023_12_16: 903
num_neurons for LPE11086/2024_01_09: 1134
num_neurons for LPE11495/2024_02_29: 1007
num_neurons for LPE11998/2024_05_08: 570
num_neurons for LPE12223/2024_06_11: 1214
num_neurons for LPE09665/2023_03_20: 563
num_neurons for LPE10883/2023_10_23: 854
num_neurons for LPE10883/2023_10_31: 1157
num_neurons for LPE10884/2023_10_12: 1071
num_neurons for LPE10885/2023_10_20: 1173
num_neurons for LPE10919/2023_11_09: 1735
num_neurons for LPE11086/2023_12_16: 903
num_neurons for LPE11086/2024_01_09: 1134
num_neurons for LPE11495/2024_02_29: 1007
num_neurons for LPE11998/2024_05_08: 570
num_neurons for LPE12223/2024_06_11: 1214
